In [3]:
import numpy as np
import pandas as pd
import re
import urllib
import json
import contextlib, unicodedata, sys
import matplotlib.pyplot as plt
import torch
import difflib
from difflib import SequenceMatcher
import os
from bs4 import B

In [4]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

In [5]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [7]:
%cd /Users/quert/Downloads/gcp_tmp/raw_text
clusters = []
with open("title_page_whole_test.txt", "r") as fread:
    for line in fread.readlines():
        clusters.append(json.loads(line.strip()))

/Users/quert/Downloads/gcp_tmp/raw_text


In [9]:
clusters[0].keys()
# title_non_update_link
# title_update_link
# update_bs4
# non_update_bs4

dict_keys(['title', 'event_time', 'title_update_time', 'event_link', 'title_update_link', 'update_bs4', 'title_non_update_link', 'title_non_update_time', 'non_update_bs4', 'article_title', 'source_news_first', 'source_news_full', 'article_record', 'update_all_paragraph_with_references', 'update_first_paragraph', 'update_all_paragraph', 'non_update_all_paragraph_with_references', 'non_update_all_paragraph', 'non_update_first_paragraph', 'update_wiki_summary', 'non_update_wiki_summary', 'wiki_portal_summary', 'citation_position', 'event_wcep_id'])

### Get the indices of required instances
* Keeep English language, with non-updated full-content existed, and meaningful updated full-content

In [13]:
# create idx list
all_idx = [i for i in range(len(clusters))]
rmved_idx = []

for idx in range(len(clusters)):
    try:
        if bool(len(clusters[idx]['non_update_all_paragraph'].split('. ')) == 1):
            rmved_idx.append(idx)
    except:
        error_idx.append(idx)
idx_list = Diff(all_idx, rmved_idx)
len(all_idx), len(idx_list)

(239, 232)

In [26]:
rmved_idx = []
for idx in idx_list:
    if len(clusters[idx]['update_first_paragraph'].split())<=10:
        rmved_idx.append(idx)
id_list = Diff(idx_list, rmved_idx)
len(idx_list), len(id_list)

(232, 226)

In [27]:
# filter by language of summ==English
en_ids = []
for idx in id_list:
    if isEnglish(clusters[idx]['wiki_portal_summary']):
        en_ids.append(idx)
len(id_list), len(en_ids)
# after filtering, we have 201 instances in test set

(226, 201)

In [135]:
en_ids[:5]

[1, 2, 4, 5, 6]

### Parse the unicode into text
* Parse the unicode into text
* Paragraphs are separate with `. \\c\\c`

In [67]:
log = []
with open("test_text.txt.src", "w") as f:
    for idx in en_ids:
        bef_rec = []
        non_update_all_paragraph = clusters[idx]['non_update_all_paragraph'].split('\n')
        bef_rec.extend(non_update_all_paragraph)
        s = '.\c'
        bef_rec_str = s.join(bef_rec).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(fwrite):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("test_text.txt.src", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("test_text.txt.src", "w") as f:
    for line in log:
        f.write(line+"\n")

In [76]:
log = []
with open("test_text.txt.tgt", "w") as f:
    for idx in en_ids:
        tgt_rec = []
        update_all_paragraph = clusters[idx]["update_all_paragraph"].split("\n")
        tgt_rec.extend(update_all_paragraph)
        s = ".\c"
        tgt_rec_str = s.join(tgt_rec).replace("\\\\\c", "\c\c")
        with contextlib.redirect_stdout(f):
            print({unicodedata.normalize('NFKD', bef_rec_str).encode('utf-8', 'ignore').decode('utf-8')})
with open("test_text.txt.tgt", "r") as f:
    for line in f.readlines():
        log.append(line.strip().replace(".\\\\c", "\c\c").replace("\\\'s", "'s").replace("{\'", "").replace("\'}", "").replace(".\\c\\c", ". \\c\\c"))
with open("test_text.txt.tgt", "w") as f:
    for line in log:
        f.write(line+"\n")

### Extract the section name and number of paragraphs under each section

In [515]:
raw_text = clusters[6]["non_update_bs4"]
# end_idx = re.findall('<span class="mw-headline" id=.+">(.*)</span>', raw_text).index("References")
# sec_names = re.findall('<span class="toctext">(.*)</span>', raw_text)[:end_idx]

main_secs = re.findall('<span class="tocnumber">\d+</span> <span class="toctext">(.*)</span>', raw_text)
all_secs = re.findall('<span class="mw-headline" id=.+">(.*)</span>', raw_text)

# Return the global index of main section
main_global_ids = []
for main_sec in main_secs:
    # The indices of main sec in all_secs
    ids = all_secs.index(main_sec)
    main_global_ids.append(ids)

# Return the parent section for each sub-section
mod_secs = all_secs.copy()
main_rec = []
for idx in range(len(all_secs)):
    for idd in main_global_ids:
        if idx==idd: 
            mod_secs[idx] = mod_secs[idx]
            main_rec.append(idd)
        else: mod_secs[idx] = all_secs[main_rec[-1]]

# Concatenate the parent section name with sub-section name
for idx in range(len(all_secs)):
    if all_secs[idx]==mod_secs[idx]: mod_secs[idx]==mod_secs[idx]
    else: mod_secs[idx] = str(mod_secs[idx]) + " " + all_secs[idx]

# Get the section names except "References"
end_idx = mod_secs.index("References")
sec_names = mod_secs[:end_idx]

# Count the #paragraphs under each section (main section and sub-section), exclude the section with zero paragraph 
num_pars = []
for i in range(len(sec_names)):
    pars_str = str(clusters[8]["non_update_bs4"].split('<span class="mw-headline')[i+1])
    num_par = len(re.split("<p>|<li>", pars_str)[1:])
    num_pars.append(num_par)
sec_pars = dict(zip(sec_names, num_pars))
secs_pars_nums = {k: v for k, v in sec_pars.items() if v!=0}

if "Notes" in secs_pars_nums.keys(): del secs_pars_nums["Notes"]
elif "See also" in secs_pars_nums.keys(): del secs_pars_nums["See also"]

secs_pars_nums

ValueError: 'References' is not in list

In [518]:
raw_text = clusters[6]["non_update_bs4"]
# end_idx = re.findall('<span class="mw-headline" id=.+">(.*)</span>', raw_text).index("References")
# sec_names = re.findall('<span class="toctext">(.*)</span>', raw_text)[:end_idx]

main_secs = re.findall('<span class="tocnumber">\d+</span> <span class="toctext">(.*)</span>', raw_text)
all_secs = re.findall('<span class="mw-headline" id=.+">(.*)</span>', raw_text)
all_secs

['Background', 'Timeline', 'Vaccination', 'References']

In [516]:
clusters[6]["title_non_update_link"]

'https://en.wikipedia.org/w/index.php?title=COVID-19_pandemic_in_Palau&oldid=1032871488'

In [347]:
error_ids = []
for idx in en_ids:
    try:
        raw_text = clusters[idx]["non_update_bs4"]
        end_idx = re.findall('<span class="toctext">(.*)</span>', raw_text).index("See also")
    except:
        error_ids.append(idx)
error_ids
# len(error_ids) # id: 2 and others
# raw_text = clusters[2]["non_update_bs4"]
# end_idx = re.findall('<span class="toctext">(.*)</span>', raw_text)
# raw_text

[2,
 6,
 8,
 13,
 25,
 28,
 33,
 34,
 35,
 36,
 47,
 48,
 68,
 71,
 78,
 80,
 82,
 83,
 85,
 86,
 89,
 90,
 91,
 94,
 103,
 104,
 105,
 107,
 117,
 118,
 119,
 120,
 124,
 128,
 130,
 131,
 132,
 133,
 134,
 137,
 138,
 141,
 144,
 146,
 149,
 155,
 156,
 157,
 168,
 172,
 176,
 180,
 182,
 190,
 193,
 194,
 195,
 196,
 199,
 200,
 209,
 210,
 213,
 215,
 216,
 217,
 218,
 219,
 221,
 222,
 227,
 230,
 231,
 236]

* Since full-article includes "summary", we have to remove the "summary" before alignment -> then add back to the text afterwards


In [152]:
# summary
# summary = clusters[2]["non_update_wiki_summary"]
contents_with_summ = []
with open("test_text.txt.src", "r") as f:
    for line in f.readlines():
        contents_with_summ.append(line)

for idx in range(len(en_ids)):
    real_id = en_ids[idx]
    s = "\\c\\c"
    summs = clusters[real_id]["non_update_wiki_summary"].split("\n") # splitted summary
    full_content_with_summ = contents_with_summ[idx]
    with_summs = full_content_with_summ.split("\\c\\c") # splitted contents with summ
    for par in with_summs:
        if par in summs: with_summs.remove(par) 
with open("test_no_summs.txt.src", "w") as f:
    for line in with_summs: f.write(line+"\n")

In [153]:
no_summs = []
with open("test_no_summs.txt.src", "r") as f:
    for line in f.readlines(): no_summs.append(line)

In [154]:
no_summs[0]

"In March and April 2021, Russia started to mass thousands of military personnel and equipment near its border with Ukraine, representing the highest force mobilization since the country's annexation of Crimea in 2014. This precipitated an international crisis and generated concerns over a potential invasion. Satellite imagery showed movements of armor, missiles, and heavy weaponry. The troops were partially removed by June. The crisis was renewed in October and November 2021, when over 100,000 Russian troops were again massed near the border by December. \n"

In [ ]:
contents_with_summ[0]